<a href="https://colab.research.google.com/github/snufkin92/colab_tutorial/blob/master/section_08/distributed_expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 分散表現の確認
word2vecによる分散表現について学びます。

## コーパスの前処理
前のセクションと同様に、コーパスに前処理を行います。

In [ ]:
import re
import pickle
from janome.tokenizer import Tokenizer

with open("wagahaiwa_nekodearu.txt", mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    wagahai_original = f.read()

wagahai = re.sub("《[^》]+》", "", wagahai_original) # ルビの削除
wagahai = re.sub("［[^］]+］", "", wagahai) # 読みの注意の削除
wagahai = re.sub("[｜ 　「」\n]", "", wagahai) # | と全角半角スペース、「」と改行の削除

seperator = "。"  # 。をセパレータに指定
wagahai_list = wagahai.split(seperator)  # セパレーターを使って文章をリストに分割する
wagahai_list.pop() # 最後の要素は空の文字列になるので、削除
wagahai_list = [x+seperator for x in wagahai_list]  # 文章の最後に。を追加

t = Tokenizer()

wagahai_words = []
for sentence in wagahai_list:
    wagahai_words.append(list(t.tokenize(sentence, wakati=True)))  # 文章ごとに単語に分割し、リストに格納

with open('wagahai_words.pickle', mode='wb') as f:  # pickleに保存
    pickle.dump(wagahai_words, f)

保存できていることを確認します。

In [ ]:
with open('wagahai_words.pickle', mode='rb') as f:
    wagahai_words = pickle.load(f)

print(wagahai_words[:10])

[['吾輩', 'は', '猫', 'で', 'ある', '。'], ['名前', 'は', 'まだ', '無い', '。'], ['どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。'], ['何', 'でも', '薄暗い', 'じめじめ', 'し', 'た', '所', 'で', 'ニャーニャー', '泣い', 'て', 'いた事', 'だけ', 'は', '記憶', 'し', 'て', 'いる', '。'], ['吾輩', 'は', 'ここ', 'で', '始め', 'て', '人間', 'という', 'もの', 'を', '見', 'た', '。'], ['しかも', 'あと', 'で', '聞く', 'と', 'それ', 'は', '書生', 'という', '人間', '中', 'で', '一番', '獰悪', 'な', '種族', 'で', 'あっ', 'た', 'そう', 'だ', '。'], ['この', '書生', 'という', 'の', 'は', '時々', '我々', 'を', '捕え', 'て', '煮', 'て', '食う', 'という', '話', 'で', 'ある', '。'], ['しかし', 'その', '当時', 'は', '何', 'という', '考', 'も', 'なかっ', 'た', 'から', '別段', '恐し', 'いとも', '思わ', 'なかっ', 'た', '。'], ['ただ', '彼', 'の', '掌', 'に', '載せ', 'られ', 'て', 'スー', 'と', '持ち上げ', 'られ', 'た', '時', '何だか', 'フワフワ', 'し', 'た', '感じ', 'が', 'あっ', 'た', 'ばかり', 'で', 'ある', '。'], ['掌', 'の', '上', 'で', '少し', '落ちつい', 'て', '書生', 'の', '顔', 'を', '見', 'た', 'の', 'が', 'いわゆる', '人間', 'という', 'もの', 'の', '見', '始', 'で', 'あろ', 'う', '。']]


## word2vecを用いた学習
今回はword2vecのためにライブラリgensimを使います。  
gensimは、様々なトピックモデルを実装したPythonライブラリです。  
トピックモデルとは、潜在的なトピックから文章が確率的に生成されると仮定したモデルです。

gensimについて、詳細は以下のリンクを参考にどうぞ。  
https://radimrehurek.com/gensim/

以下では、word2vecを用いてコーパスの学習を行い、学習済みのモデルを作成します。

In [ ]:
print(type(wagahai_words))
wagahai_words[:3]

<class 'list'>


[['吾輩', 'は', '猫', 'で', 'ある', '。'],
 ['名前', 'は', 'まだ', '無い', '。'],
 ['どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。']]

In [ ]:
# scipy=1.12.0を使わないとImportError: cannot import name 'triu' from 'scipy.linalg' が発生
from gensim.models import word2vec

# vector_size : 中間層のニューロン数
# min_count : この値以下の出現回数の単語を無視
# window : 対象単語を中心とした前後の単語数
# epochs : epochs数
# sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
model = word2vec.Word2Vec(wagahai_words,
                          vector_size=100,
                          min_count=5,
                          window=5,
                          epochs=20,
                          sg = 0)

分散表現を見ていきましょう。  
重みを表す行列（分散表現）の形状と、行列そのものを表示します。

In [ ]:
print(model.wv.vectors.shape)  # 分散表現の形状
print(model.wv.vectors)  # 分散表現

(3309, 100)
[[-0.13262224  0.6376691   0.9703918  ... -0.83940995  0.6842959
   0.5759229 ]
 [-0.8580033  -0.66843766  1.4828197  ... -0.47005606 -0.33892664
  -0.17183577]
 [ 0.47351834  0.24001913 -1.0540961  ... -0.4554502   0.34211117
  -0.03961928]
 ...
 [ 0.03896393  0.06777891  0.03392925 ... -0.12314384 -0.0132106
  -0.06999081]
 [-0.16052945  0.11881156  0.15593597 ... -0.11861178  0.13037367
  -0.06686825]
 [-0.08516419 -0.06244256 -0.00626748 ... -0.1691904   0.04974204
  -0.0372024 ]]


3309個の単語が100次元で表現されている

語彙の数、および語彙の最初の10語を表示します。

In [ ]:
print(len(model.wv.index_to_key))  # 語彙の数
print(model.wv.index_to_key[:10])  # 最初の10単語

3309
['の', '。', 'て', '、', 'は', 'に', 'を', 'と', 'が', 'た']


語彙における、最初の単語の単語ベクトルを2通りの方法で表示します。

In [ ]:
print(model.wv.vectors[0])  # 最初のベクトル

[-1.32622242e-01  6.37669086e-01  9.70391810e-01  6.19177222e-01
 -8.04323852e-01  1.02501303e-01  5.10776937e-01  4.11223263e-01
 -5.13011396e-01 -1.30582391e-03  2.30451003e-01  1.74877167e-01
 -6.38507962e-01  4.77089971e-01 -5.17559469e-01  1.82515487e-01
 -4.56721663e-01 -6.80352569e-01 -2.65191525e-01  4.51699823e-01
  7.00099647e-01 -9.87528637e-02  3.00687373e-01  1.08244693e+00
 -3.18306476e-01  2.11888433e-01 -4.26928043e-01  9.27322954e-02
  1.02046825e-01 -5.75903237e-01 -1.27754486e+00  7.90337250e-02
  3.15923765e-02 -3.62777978e-01  6.08772457e-01  6.56754255e-01
 -1.62177220e-01 -7.44768500e-01 -1.63632959e-01 -7.74770200e-01
 -9.56861615e-01 -3.66354227e-01 -1.25183713e+00  5.17315924e-01
  5.71118116e-01  5.95237970e-01 -5.19761503e-01 -2.89529622e-01
  2.58446991e-01  7.28476703e-01 -1.95298623e-02  7.87290215e-01
 -7.55575657e-01  7.69167244e-01 -5.73947489e-01 -4.16455328e-01
 -3.53526503e-01 -8.58177423e-01 -8.32668304e-01  2.38506012e-02
 -8.91649246e-01 -2.39230

In [ ]:
print(model.wv.__getitem__("の"))  # 最初の単語「の」のベクトル

[-1.32622242e-01  6.37669086e-01  9.70391810e-01  6.19177222e-01
 -8.04323852e-01  1.02501303e-01  5.10776937e-01  4.11223263e-01
 -5.13011396e-01 -1.30582391e-03  2.30451003e-01  1.74877167e-01
 -6.38507962e-01  4.77089971e-01 -5.17559469e-01  1.82515487e-01
 -4.56721663e-01 -6.80352569e-01 -2.65191525e-01  4.51699823e-01
  7.00099647e-01 -9.87528637e-02  3.00687373e-01  1.08244693e+00
 -3.18306476e-01  2.11888433e-01 -4.26928043e-01  9.27322954e-02
  1.02046825e-01 -5.75903237e-01 -1.27754486e+00  7.90337250e-02
  3.15923765e-02 -3.62777978e-01  6.08772457e-01  6.56754255e-01
 -1.62177220e-01 -7.44768500e-01 -1.63632959e-01 -7.74770200e-01
 -9.56861615e-01 -3.66354227e-01 -1.25183713e+00  5.17315924e-01
  5.71118116e-01  5.95237970e-01 -5.19761503e-01 -2.89529622e-01
  2.58446991e-01  7.28476703e-01 -1.95298623e-02  7.87290215e-01
 -7.55575657e-01  7.69167244e-01 -5.73947489e-01 -4.16455328e-01
 -3.53526503e-01 -8.58177423e-01 -8.32668304e-01  2.38506012e-02
 -8.91649246e-01 -2.39230

両者ともに同じベクトルですね。

## 課題:
単語「猫」を単語ベクトルで表してみましょう。  
また、skip-gramも試して結果を比較してみましょう。

In [ ]:
print(model.wv.__getitem__("猫"))

[ 0.49389535 -0.5368431   0.01848227 -0.2998496   0.14273547  0.8160949
 -0.7390623   0.9594122   1.2501879  -0.04962321  0.76159286  1.0826745
 -0.22315796  0.08589792 -0.31518108  0.09116571  0.18727177 -0.3275528
  0.6151032  -0.02364909  0.7297533   0.11873629  1.0472162  -0.24857353
  0.32181436  0.6682793  -1.004732   -0.5469218   0.71741164  0.77665603
 -1.8105392  -0.20079927 -0.05320627 -1.0941718   0.37810582  0.45051005
  0.41972822 -0.54603505 -0.08725882 -0.22658247 -1.0787129  -0.32814866
  0.34240833  0.3828128  -0.19413543 -0.04672657  0.7432857  -1.2522031
 -0.09242587 -0.25035205 -1.7908078   0.6343977  -0.3772061   0.1019305
  1.3225694  -0.58262074 -0.06361256 -0.7413524   0.9291293   1.8064713
  0.4359546  -0.29224864 -0.09469255 -0.3409965  -1.2350273  -0.26646197
 -0.04862994  1.5623655  -0.56125766  0.66185653  0.7274623  -0.22990547
  0.784491   -0.15230495  0.21330275 -0.87824094  0.0054565   0.46843407
 -0.64842165 -0.5134241  -0.6845294   0.62715876 -0.52073

In [ ]:
from gensim.models import word2vec

model_skip = word2vec.Word2Vec(wagahai_words,
                          vector_size=100,
                          min_count=5,
                          window=5,
                          epochs=20,
                          sg = 1)

In [ ]:
print(model_skip.wv.vectors.shape)  # 分散表現の形状

# 猫の分散表現（CBOWと違う！）
print(model_skip.wv.__getitem__("猫"))

(3309, 100)
[-0.08673591  0.30820256 -0.02953538  0.13294083 -0.36908588  0.7193044
  0.06363755  0.66458195  0.71625227  0.24718437  1.0639889   0.28868714
  0.05032108  0.32984638 -0.1912864   0.06336804  0.22127911 -0.2506569
  0.2996364  -0.2892944   0.3804191   0.22440371 -0.3426052   0.84506214
  0.01301781  0.06392673 -0.96968764 -0.30992478  0.11135981  0.42982787
 -0.45565283 -0.14300655 -0.6164357  -0.6803418  -0.12909864  0.49735367
  0.36628085 -0.04091739 -0.3316552  -0.2056428   0.26770827  0.5224419
  0.12160255 -0.0652353  -0.28884235  0.5131209   0.45722276 -0.5391972
  0.58661693  0.15865748 -0.22856441 -0.20202272 -0.5046227  -0.31021357
 -0.4292442   0.13184759  0.10953931 -0.02671878 -0.0860863   0.5744976
  0.14074619 -0.2067359  -0.40397412 -0.0808498  -0.5287643   0.415613
 -0.14085338  1.2002153  -0.49014428  0.2723026  -0.0326972   0.43525562
  0.02252262  0.42087543 -0.04105791  0.08798922  0.15534075  0.22250989
 -0.46786666 -0.26777902  0.3262293  -0.080352

猫の分散表現がCBOWと違う！

In [ ]:
print(model_skip.wv.__getitem__("黒"))

[-0.22383127  0.2792024  -0.47555608  0.52744585 -0.11830074  0.22855282
  0.72555864  0.32415372  0.16922261 -0.2024026  -0.4543991  -0.18941857
  0.32629433  0.1830218  -0.16302624  0.01073745  0.0799717  -0.11806955
  0.14646496 -0.36892864 -0.1309027   0.2595161   0.03119702  0.40649137
 -0.36538404 -0.08947231 -0.08917995  0.28545022  0.23824848  0.35414687
 -0.360418    0.2725609   0.09482723 -0.18300706 -0.03456452  0.28644374
 -0.1051873  -0.19834138  0.04621578 -0.37643468  0.22198312  0.02431218
 -0.3601935  -0.36572218  0.31109503  0.68825245 -0.20120008 -0.30870935
  0.31606176 -0.18483147  0.0857709  -0.06918326 -0.7218825   0.02768981
  0.08873706  0.10635679 -0.17264205 -0.59647065 -0.14882423 -0.20247896
 -0.14408056 -0.01363565  0.15590867 -0.337324   -0.05907686  0.32930323
 -0.16766925  0.8017681  -0.17871675  0.28338227  0.0134429  -0.15083429
  0.18113148 -0.38019818  0.10878071  0.08877594  0.13210435  0.17675166
 -0.14678341 -0.28720695  0.41396576 -0.09059002 -0

In [ ]:
# 黒猫という単語は学習していないからエラー
print(model_skip.wv.__getitem__("黒猫"))

KeyError: "Key '黒猫' not present"